First we're going to load the necessary libraries for training out model

In [0]:
import os
import numpy as np
import cv2
import time
import random
import matplotlib.pyplot as plt
from random import shuffle
from tqdm import tqdm
from imgaug import augmenters as iaa
from keras.models import Model, load_model
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, CSVLogger, LambdaCallback

from keras.applications import MobileNetV2
from keras.applications.densenet import DenseNet201



Next we're going to set which model we'll be training. This is up to you and your application. From our work we recommend DenseNet201 for accuracy, and MobileNetv2 for efficiency. Replace 'DenseNet201' with 'MobileNetV2' if you'd like to try it.

In [0]:
Network = 'DenseNet201'

We're going to set the default settings for the network

In [0]:
TRAIN_DIR = 'Train/'
TEST_DIR = 'Test/'
IMG_SIZE = 224
epoch = 200
batch_size = 32
train_all_weights = True

Now we're going to gather information about the data including: The number of images, the maximum number of a classification, the ratio balance, and store it in a dictionary

In [31]:
Class_List = os.listdir(TRAIN_DIR)
Num_Classes = len(Class_List)

Num_Images_Dict, Num_Test_Images_Dict, ratio_dict = {}, {}, {}

for animal in os.listdir(TRAIN_DIR): 
  Num_Images_Dict[animal] = len(os.listdir(TRAIN_DIR + animal))
for animal in os.listdir(TEST_DIR): 
  Num_Test_Images_Dict[animal] = len(os.listdir(TEST_DIR + animal))

for classification in Num_Images_Dict:
  if Num_Images_Dict[classification] > maxbreak: Num_Images_Dict[classification] = maxbreak

max_class_num = Num_Images_Dict[max(Num_Images_Dict, key=Num_Images_Dict.get)]

for classification in Num_Images_Dict:
  ratio_dict[classification] = Num_Images_Dict[classification] / max_class_num


FileNotFoundError: ignored

Here we define to image augmentation strategies. The first when building the data, the second when training.

In [0]:
train_aug = iaa.SomeOf((1, 3), [  # Random number between 0, 3
    iaa.Fliplr(0.5),  # Horizontal flips                     
    iaa.Add((-5, 5)),  # Overall Brightness                   
    iaa.Multiply((0.95, 1.05), per_channel=0.2),  # Brightness multiplier per channel    0.05
    iaa.Sharpen(alpha=(0.1, 0.75), lightness=(0.85, 1.15)),  # Sharpness                            0.05
    iaa.WithColorspace(to_colorspace='HSV', from_colorspace='RGB',  # Random HSV increase                  0.09
                       children=iaa.WithChannels(0, iaa.Add((-30, 30)))),
    iaa.WithColorspace(to_colorspace='HSV', from_colorspace='RGB',
                       children=iaa.WithChannels(1, iaa.Add((-30, 30)))),
    iaa.WithColorspace(to_colorspace='HSV', from_colorspace='RGB',
                       children=iaa.WithChannels(2, iaa.Add((-30, 30)))),
    iaa.AddElementwise((-10, 10)),  # Per pixel addition                   0.11
    iaa.CoarseDropout((0.0, 0.02), size_percent=(0.02, 0.25)),  # Add large black squares              0.13
    iaa.GaussianBlur(sigma=(0.1, 1.0)),  # GaussianBlur                         0.14
    iaa.Grayscale(alpha=(0.1, 1.0)),  # Random Grayscale conversion          0.17
    iaa.Dropout(p=(0, 0.1), per_channel=0.2),  # Add small black squares              0.17
    iaa.AdditiveGaussianNoise(scale=(0.0, 0.05 * 255), per_channel=0.5),
    # Add Gaussian per pixel noise         0.26
    iaa.ElasticTransformation(alpha=(0, 1.0), sigma=0.25),  # Distort image by rearranging pixels  0.70
    iaa.ContrastNormalization((0.75, 1.5)),  # Contrast Normalization               0.95
    iaa.weather.Clouds(),
    iaa.weather.Fog(),
    iaa.weather.Snowflakes()], 
    random_order=True)

batch_aug = iaa.SomeOf((1, 2), [
  iaa.Affine(scale={"x": (0.8, 1.2), "y": (0.8, 1.2)},  # Affine: Scale/zoom,                  
             translate_percent={"x": (-0.2, 0.2), "y": (-0.2, 0.2)},  # Translate/move
             rotate=(-90, 90), shear=(-4, 4)),  # Rotate and Shear
  iaa.PiecewiseAffine(scale=(0, 0.05)),],  # Distort Image similar water droplet  
  random_order=True)

Next we're going to define a function building our training and testing data. This goes through each image, resizes them so they can fit on the GPU and returns a list of them all. It also provides augmentation and the described ratio prior distribution.

In [0]:
def create_data(DATA_DIR, IMG_SIZE, Train):
    data, classnum = [], -1
    for animal in tqdm(os.listdir(DATA_DIR)[:5]):
        print('\n', animal)
        classnum += 1
        i, imglist = 0, os.listdir(DATA_DIR + animal)
        shuffle(imglist)
        for i in range(len(imglist)-1):
            img = cv2.resize(cv2.imread(DATA_DIR + animal + '/' + imglist[i]), (IMG_SIZE, IMG_SIZE))
            zerolist = [0] * Num_Classes
            zerolist[classnum] += 1
            data.append([np.array(img), zerolist])
            if i >= maxbreak: break
            if Train == True:
                for k in range(max_class_num - len(imglist)):
                    aug_img = train_aug.augment_image(img)
                    zerolist = [0] * Num_Classes
                    zerolist[classnum] += 1
                    data.append([np.array(aug_img), zerolist])
    return data

Next we're going to create the data using the create_data function and convert it into our data and labels for our train and test sets. Since we have the data and labels isolated now, we can delete the collection of them to save memory

In [0]:
train_data = create_data(TRAIN_DIR, IMG_SIZE, Train=True)
test_data = create_data(TEST_DIR, IMG_SIZE, Train=False)

print('Train Size: {}'.format(len(train_data)))
print('Test Size: {}'.format(len(test_data)))

x_train = np.array([i[0] for i in train_data]).reshape(-1, IMG_SIZE, IMG_SIZE, 3)
y_train = np.array([i[1] for i in train_data])

x_test = np.array([i[0] for i in test_data]).reshape(-1, IMG_SIZE, IMG_SIZE, 3)
y_test = np.array([i[1] for i in test_data])

del train_data
del test_data

x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0

We're not going to write a simple text file that provides info of the training test distribution. We're going to write a simple plotting function to view our training and testing distribution. These should be as similar as possible. I also define a plotting metrics function for our results that we use later.

In [0]:
def Write_Classifications():
    file = open('{}/{}/{}_Class_{}_saved_models/{}_{}_{}_Classes.txt'.format(Network, IMG_SIZE, Num_Classes, starttime,
                                                                          Num_Classes, IMG_SIZE, starttime), 'w')
    file.write('Classifications and Training Numbers:\n\n')
    for animal in Num_Images_Dict: file.write(str(animal) + ':' + str(Num_Images_Dict[animal]) + '\n')
    file.write('\n\nClassifications and Testing Numbers:\n\n')
    for animal in Num_Test_Images_Dict: file.write(str(animal) + ':' + str(Num_Test_Images_Dict[animal]) + '\n')


def Plot_Data_Distribution(image_dict, data_type):
    plt.title('{} Class Training Distribution'.format(Num_Classes))
    plt.ylabel('Num Images')
    plt.bar(image_dict.keys(), image_dict.values(), color='g')
    plt.xticks(rotation=90)
    plt.savefig('{}/{}/{}_Class_{}_saved_models/{}_Distribution.png'.format(Network, IMG_SIZE, Num_Classes,
                                                                            starttime, data_type))

def plot_metrics(data1, data2, IMG_SIZE, metric):
    plt.plot(data1)
    plt.plot(data2)
    plt.title('History of Multiclass Animal Model {} During Training'.format(metric))
    plt.ylabel(metric)
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Test'], loc='upper left')
    plt.savefig('{}/{}/{}_Class_{}_saved_models/{}_{}_{}_{}.png'.format(Network, IMG_SIZE, Num_Classes,
                                                            starttime, Num_Classes, IMG_SIZE, starttime, metric))
    plt.clf()
    
Write_Classifications()
Plot_Data_Distribution(Num_Images_Dict, 'Training')
Plot_Data_Distribution(Num_Test_Images_Dict, 'Testing')

Next we're going to load our network depending on if you selected 'DenseNet201' or 'MobileNetV2' previously. We're also going to print a summary of the network, select the optimizer Adam, and create a csv log of the training process. 

In [0]:

shape = (IMG_SIZE, IMG_SIZE, 3)
if network == 'DenseNet201':
    base_model = DenseNet201(input_shape=shape, weights='imagenet', include_top=False)
elif network == 'MobileNetV2':
    base_model = MobileNetV2(input_shape=shape, weights='imagenet', include_top=False)
else:
    print('network name not either DenseNet201 or MobileNetV2')
x = base_model.output
x = GlobalAveragePooling2D(name='ex_Pool')(x)
x = Dense(1024, activation='relu', name='ex_Dense1', kernel_initializer='glorot_normal')(x)
x = Dense(1024, activation='relu', name='ex_Dense2', kernel_initializer='glorot_normal')(x)
x = Dense(512, activation='relu', name='ex_Dense3', kernel_initializer='glorot_normal')(x)
output = Dense(Num_Classes, activation='softmax', name='softmax')(x)
for layer in base_model.layers: 
  layer.trainable = train_all_weights
model = Model(inputs=base_model.input, outputs=output)
model.summary()
model.compile(optimizer=Adam(lr=0.0001, decay=1e-6),
              loss={'softmax': 'categorical_crossentropy'},
              metrics={'softmax': 'accuracy'})

csv_logger = CSVLogger('{}/{}/{}_Class_{}_saved_models/{}_{}_{}_Logger.csv'.format(Network, IMG_SIZE, Num_Classes,
                                                                starttime, Num_Classes, IMG_SIZE, starttime), separator=',')

Here we're going to define a generator, which allows us to modify the input images with augmentation and ratio prior for each epoch of training. 

In [0]:


def generator(x_train, y_train, batch_size):
    batch_x_train = np.zeros((batch_size, IMG_SIZE, IMG_SIZE, 3))
    batch_y_train = np.zeros((batch_size, Num_Classes))
    index_dict = {}
    for i in range(len(x_train)):
        if y_train[i].argmax() not in index_dict: index_dict[y_train[i].argmax()] = [i]
        else: index_dict[y_train[i].argmax()].append(i)

    while True:
        i = 0
        while i < batch_size - 1:
            index = np.random.randint(0, len(x_train) - 1)
            while True:
                if random.random() > ratio_dict[Class_List[y_train[index].argmax()]]:
                    random_index = random.choice(index_dict[y_train[index].argmax()])
                    batch_x_train[i] = batch_aug.augment_image(x_train[random_index])
                    batch_y_train[i] = y_train[random_index]
                    i += 1
                    if i == batch_size - 1: break
                else:
                    batch_x_train[i] = batch_aug.augment_image(x_train[index])
                    batch_y_train[i] = y_train[index]
                    break
            i += 1
        yield batch_x_train, batch_y_train

Finally now, we train the model! After the model has finished training (which can take a looong while, possibly days with a large enough dataset). We'll also plot our metrics.

In [0]:
history = model.fit_generator(generator(x_train, y_train, batch_size),
                                  validation_data=(x_test, y_test),
                                  shuffle=True,
                                  steps_per_epoch=x_train.shape[0] / batch_size,
                                  epochs=epochs,
                                  callbacks=[csv_logger,
                                      ModelCheckpoint('%s/%s/%s_Class_%s_saved_models/weights.{epoch:02d}-'
                                                     '{val_acc:.2f}.hdf5' % (Network, IMG_SIZE, Num_Classes, starttime),
                                                      monitor='val_acc', verbose=0, save_best_only=True,
                                                      save_weights_only=False, mode='auto', period=1)])


And that's it! Hopefully this was helpful for training a model. 